<a href="https://colab.research.google.com/github/robsonbfreitas/TrabalhoFinal/blob/main/TrabalhoFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Importando bibliotecas
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv('features.csv')
#dftest = pd.read_csv('test.csv')
dftrain = pd.read_csv('train.csv')
df.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [3]:
dftrain.head(30)

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False
5,1,1,2010-03-12,21043.39,False
6,1,1,2010-03-19,22136.64,False
7,1,1,2010-03-26,26229.21,False
8,1,1,2010-04-02,57258.43,False
9,1,1,2010-04-09,42960.91,False


In [4]:
df_teste = dftrain.query('Store >= 23')
df_train = dftrain.query('Store < 23')
df_teste.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
216819,23,1,2010-02-05,32452.79,False
216820,23,1,2010-02-12,50360.71,True
216821,23,1,2010-02-19,28504.70,False
216822,23,1,2010-02-26,23033.73,False
216823,23,1,2010-03-05,29662.30,False


In [5]:
df_teste.shape

(204751, 5)

In [6]:
df_train.shape

(216819, 5)

In [ ]:
df_train.tail()

,Store,Dept,Date,Weekly_Sales,IsHoliday
216814,22,98,2012-10-05,427.25,False
216815,22,98,2012-10-12,247.46,False
216816,22,98,2012-10-19,390.40,False
216817,22,98,2012-10-26,229.21,False
216818,22,99,2010-09-24,27.15,False


In [7]:
#frames = [df, dftrain]
df_testeFeature = df_teste.merge(df, on=['Store','Date'], how='left')
df_trainWithFeature = df_train.merge(df, on=['Store','Date'], how='left')

In [8]:
df_trainWithFeature.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y
0,1,1,2010-02-05,24924.50,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,1,2010-02-12,46039.49,True,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,1,2010-02-19,41595.55,False,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,1,2010-02-26,19403.54,False,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,1,2010-03-05,21827.90,False,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [ ]:
df.query('Date == "2010-02-12"').query('Store == 1')

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.24217,8.106,True


In [ ]:
dftrain.query('Date == "2010-02-12"').query('Store == 1')

,Store,Dept,Date,Weekly_Sales,IsHoliday
1,1,1,2010-02-12,46039.49,True
144,1,2,2010-02-12,44682.74,True
287,1,3,2010-02-12,10887.84,True
430,1,4,2010-02-12,35351.21,True
573,1,5,2010-02-12,29620.81,True
...,...,...,...,...,...
9371,1,93,2010-02-12,70202.02,True
9514,1,94,2010-02-12,62329.48,True
9657,1,95,2010-02-12,111390.36,True
9922,1,97,2010-02-12,33874.47,True


In [ ]:
df2.query('Date == "2010-02-12"').query('Store == 1')

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y
1,1,1,2010-02-12,46039.49,True,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.24217,8.106,True
144,1,2,2010-02-12,44682.74,True,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.24217,8.106,True
287,1,3,2010-02-12,10887.84,True,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.24217,8.106,True
430,1,4,2010-02-12,35351.21,True,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.24217,8.106,True
573,1,5,2010-02-12,29620.81,True,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.24217,8.106,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9371,1,93,2010-02-12,70202.02,True,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.24217,8.106,True
9514,1,94,2010-02-12,62329.48,True,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.24217,8.106,True
9657,1,95,2010-02-12,111390.36,True,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.24217,8.106,True
9922,1,97,2010-02-12,33874.47,True,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.24217,8.106,True


In [11]:
df_XTrain = df_trainWithFeature[['Store','Date','IsHoliday_y', 'Dept', 'Weekly_Sales','MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5']]
df_XTrain.head()

,Store,Date,IsHoliday_y,Dept,Weekly_Sales,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5
0,1,2010-02-05,False,1,24924.50,NaN,NaN,NaN,NaN,NaN
1,1,2010-02-12,True,1,46039.49,NaN,NaN,NaN,NaN,NaN
2,1,2010-02-19,False,1,41595.55,NaN,NaN,NaN,NaN,NaN
3,1,2010-02-26,False,1,19403.54,NaN,NaN,NaN,NaN,NaN
4,1,2010-03-05,False,1,21827.90,NaN,NaN,NaN,NaN,NaN


In [12]:
 df_XTeste = df_testeFeature[['Store','Date','IsHoliday_y', 'Dept', 'Weekly_Sales','MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5']]
df_XTeste.head()

,Store,Date,IsHoliday_y,Dept,Weekly_Sales,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5
0,23,2010-02-05,False,1,32452.79,NaN,NaN,NaN,NaN,NaN
1,23,2010-02-12,True,1,50360.71,NaN,NaN,NaN,NaN,NaN
2,23,2010-02-19,False,1,28504.70,NaN,NaN,NaN,NaN,NaN
3,23,2010-02-26,False,1,23033.73,NaN,NaN,NaN,NaN,NaN
4,23,2010-03-05,False,1,29662.30,NaN,NaN,NaN,NaN,NaN


In [13]:
df_XTrain['Date'] = pd.to_datetime(df_XTrain['Date'])
df_XTeste['Date'] = pd.to_datetime(df_XTeste['Date'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
df_XTrain.set_index(['Date'], inplace=True)
df_XTeste.set_index(['Date'], inplace=True)

In [15]:
df_XTrain.head()

,Store,IsHoliday_y,Dept,Weekly_Sales,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5
Date,,,,,,,,,
2010-02-05,1,False,1,24924.50,NaN,NaN,NaN,NaN,NaN
2010-02-12,1,True,1,46039.49,NaN,NaN,NaN,NaN,NaN
2010-02-19,1,False,1,41595.55,NaN,NaN,NaN,NaN,NaN
2010-02-26,1,False,1,19403.54,NaN,NaN,NaN,NaN,NaN
2010-03-05,1,False,1,21827.90,NaN,NaN,NaN,NaN,NaN


In [16]:
df_XTeste.head()

,Store,IsHoliday_y,Dept,Weekly_Sales,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5
Date,,,,,,,,,
2010-02-05,23,False,1,32452.79,NaN,NaN,NaN,NaN,NaN
2010-02-12,23,True,1,50360.71,NaN,NaN,NaN,NaN,NaN
2010-02-19,23,False,1,28504.70,NaN,NaN,NaN,NaN,NaN
2010-02-26,23,False,1,23033.73,NaN,NaN,NaN,NaN,NaN
2010-03-05,23,False,1,29662.30,NaN,NaN,NaN,NaN,NaN


In [17]:
df_XTrain.fillna(0, inplace=True)
df_XTeste.fillna(0, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [19]:
df_XTeste.tail(10000)

,Store,IsHoliday_y,Dept,Weekly_Sales,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5
Date,,,,,,,,,
2011-05-27,44,False,96,3138.26,0.00,0.00,0.00,0.00,0.00
2011-06-03,44,False,96,2921.07,0.00,0.00,0.00,0.00,0.00
2011-06-10,44,False,96,3296.00,0.00,0.00,0.00,0.00,0.00
2011-06-17,44,False,96,3091.16,0.00,0.00,0.00,0.00,0.00
2011-06-24,44,False,96,3296.80,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...
2012-09-28,45,False,98,508.37,4556.61,20.64,1.50,1601.01,3288.25
2012-10-05,45,False,98,628.10,5046.74,0.00,18.82,2253.43,2340.01
2012-10-12,45,False,98,1061.02,1956.28,0.00,7.89,599.32,3990.54


In [ ]:
y = df_XTrain[['Weekly_Sales']]
y

,Weekly_Sales
Date,
2010-02-05,24924.50
2010-02-12,46039.49
2010-02-19,41595.55
2010-02-26,19403.54
2010-03-05,21827.90
...,...
2012-10-05,427.25
2012-10-12,247.46
2012-10-19,390.40


In [20]:
y = df_XTeste[['Weekly_Sales']]
y

,Weekly_Sales
Date,
2010-02-05,32452.79
2010-02-12,50360.71
2010-02-19,28504.70
2010-02-26,23033.73
2010-03-05,29662.30
...,...
2012-09-28,508.37
2012-10-05,628.10
2012-10-12,1061.02


In [21]:
y = y.iloc[1:]
#y = y.dropna()
y.head()

,Weekly_Sales
Date,
2010-02-12,50360.71
2010-02-19,28504.70
2010-02-26,23033.73
2010-03-05,29662.30
2010-03-12,33445.02


In [ ]:
X = df_XTrain[['Store','Dept','IsHoliday_y','Weekly_Sales','MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5']].shift(1)
#X = df.shift(1)
X

,Store,Dept,IsHoliday_y,Weekly_Sales,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5
Date,,,,,,,,,
2010-02-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-02-12,1.0,1.0,False,24924.50,0.00,0.00,0.00,0.00,0.00
2010-02-19,1.0,1.0,True,46039.49,0.00,0.00,0.00,0.00,0.00
2010-02-26,1.0,1.0,False,41595.55,0.00,0.00,0.00,0.00,0.00
2010-03-05,1.0,1.0,False,19403.54,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...
2012-10-05,22.0,98.0,False,136.04,4269.80,23.73,1.72,2763.63,2130.39
2012-10-12,22.0,98.0,False,427.25,5294.37,0.00,4.25,1447.19,3652.84
2012-10-19,22.0,98.0,False,247.46,1558.08,0.00,10.35,485.20,4291.26


In [22]:
X = df_XTeste[['Store','Dept','IsHoliday_y','Weekly_Sales','MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5']].shift(1)
X

,Store,Dept,IsHoliday_y,Weekly_Sales,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5
Date,,,,,,,,,
2010-02-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-02-12,23.0,1.0,False,32452.79,0.00,0.00,0.00,0.00,0.00
2010-02-19,23.0,1.0,True,50360.71,0.00,0.00,0.00,0.00,0.00
2010-02-26,23.0,1.0,False,28504.70,0.00,0.00,0.00,0.00,0.00
2010-03-05,23.0,1.0,False,23033.73,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...
2012-09-28,45.0,98.0,False,467.30,8452.20,92.28,63.24,2376.38,8670.40
2012-10-05,45.0,98.0,False,508.37,4556.61,20.64,1.50,1601.01,3288.25
2012-10-12,45.0,98.0,False,628.10,5046.74,0.00,18.82,2253.43,2340.01


In [23]:
X = X.iloc[1:]
#X = X.dropna()
X.head()

,Store,Dept,IsHoliday_y,Weekly_Sales,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5
Date,,,,,,,,,
2010-02-12,23.0,1.0,False,32452.79,0.0,0.0,0.0,0.0,0.0
2010-02-19,23.0,1.0,True,50360.71,0.0,0.0,0.0,0.0,0.0
2010-02-26,23.0,1.0,False,28504.70,0.0,0.0,0.0,0.0,0.0
2010-03-05,23.0,1.0,False,23033.73,0.0,0.0,0.0,0.0,0.0
2010-03-12,23.0,1.0,False,29662.30,0.0,0.0,0.0,0.0,0.0


In [24]:
y.shape, X.shape

((204750, 1), (204750, 9))

In [25]:
X = pd.get_dummies(X)
X.head()

,Store,Dept,Weekly_Sales,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,IsHoliday_y_False,IsHoliday_y_True
Date,,,,,,,,,,
2010-02-12,23.0,1.0,32452.79,0.0,0.0,0.0,0.0,0.0,1,0
2010-02-19,23.0,1.0,50360.71,0.0,0.0,0.0,0.0,0.0,0,1
2010-02-26,23.0,1.0,28504.70,0.0,0.0,0.0,0.0,0.0,1,0
2010-03-05,23.0,1.0,23033.73,0.0,0.0,0.0,0.0,0.0,1,0
2010-03-12,23.0,1.0,29662.30,0.0,0.0,0.0,0.0,0.0,1,0


In [26]:
X.isna().mean()

Store                0.0
Dept                 0.0
Weekly_Sales         0.0
MarkDown1            0.0
MarkDown2            0.0
MarkDown3            0.0
MarkDown4            0.0
MarkDown5            0.0
IsHoliday_y_False    0.0
IsHoliday_y_True     0.0
dtype: float64

In [27]:
reg = LinearRegression()
reg.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [28]:
X.shape

(204750, 10)

In [29]:
y.shape

(204750, 1)

In [30]:
result = reg.predict(X)

In [31]:
result

array([[31169.18551172],
       [47219.16055192],
       [27456.56631477],
       ...,
       [ 1889.80540698],
       [ 2267.80468028],
       [ 1932.91510534]])

In [32]:
# Utilizando R2 para medir a performance do modelo
r2_score(y, result)

0.8908924042500537